## Packages Used

In [22]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta


### CSV File Import

In [23]:
name = 'Workout Progress Data - master.csv'
workout_data = pd.read_csv(name)

## Exploring Data

In [24]:
# Printing the first 5 rows
workout_data.head()

,Date,Exercise,Muscle_group,Sets,Reps,Weight,Duration,Note,Unnamed: 8,Unnamed: 9
0,04/04/2022,Dumbbell Shoulder Press,Shoulders,1.0,10.0,40.0,NaN,NaN,NaN,NaN
1,04/04/2022,Dumbbell Shoulder Press,Shoulders,2.0,10.0,40.0,NaN,NaN,NaN,NaN
2,04/04/2022,Dumbbell Shoulder Press,Shoulders,3.0,10.0,40.0,NaN,NaN,NaN,NaN
3,04/04/2022,Dumbbell Bench Press,Chest,1.0,10.0,30.0,NaN,NaN,NaN,NaN
4,04/04/2022,Dumbbell Bench Press,Chest,2.0,10.0,30.0,NaN,NaN,NaN,NaN


In [25]:
# Printing the last 5 rows
workout_data.tail()

,Date,Exercise,Muscle_group,Sets,Reps,Weight,Duration,Note,Unnamed: 8,Unnamed: 9
11006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# Data shape
workout_data.shape

(11011, 10)

In [27]:
# Printing data columns
workout_data.columns
WORKOUT_DATA_COLUMNS = ['Date', 'Exercise', 'Muscle_group', 'Sets', 'Reps', 'Weight',
       'Duration', 'Note']

workout_data = workout_data[WORKOUT_DATA_COLUMNS]

In [28]:
# Dara more information
workout_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11011 entries, 0 to 11010
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          1813 non-null   object 
 1   Exercise      1813 non-null   object 
 2   Muscle_group  1808 non-null   object 
 3   Sets          1770 non-null   float64
 4   Reps          1770 non-null   float64
 5   Weight        1225 non-null   float64
 6   Duration      1 non-null      object 
 7   Note          106 non-null    object 
dtypes: float64(3), object(5)
memory usage: 688.3+ KB


## Fields

In [29]:
# Date Parameters -- Calculations
today = date.today()
startOfWeek = today - relativedelta(days=today.weekday())
endOfWeek = startOfWeek + relativedelta(days=6)
firstDayOfMonth = today.replace(day=1)
nextMonthDate = today + relativedelta(months=1, day=1)

# Formatting into string
today = today.strftime('%m-%d-%Y')
startOfWeek = startOfWeek.strftime('%m-%d-%Y')
end_of_week = endOfWeek.strftime('%m-%d-%Y')
firstDayOfMonth = firstDayOfMonth.strftime('%m-%d-%Y')
nextMonthDate = nextMonthDate.strftime('%m-%d-%Y')

# Output
message = f""""
Today: {str(today)} \n
Week Start: {str(startOfWeek)}
Week End: {str(end_of_week)} \n
Month Start: {str(firstDayOfMonth)}
Next Month Start: {str(nextMonthDate)}
"""
print(message)

"
Today: 01-03-2023 

Week Start: 01-02-2023
Week End: 01-08-2023 

Month Start: 01-01-2023
Next Month Start: 02-01-2023



### Sorting Data

In [30]:
workout_data = workout_data.sort_values(by=['Date','Exercise','Sets'])

In [31]:
# Unique days count
unique_days_list = pd.DataFrame(workout_data['Date'].unique(), columns=['Date'])
print(f"Count of unique days in all csv: {len(unique_days_list)}")


Count of unique days in all csv: 124


---

### Current Month Data
Querying all data related to the current month

In [32]:
# current_month_data = workout_data[workout_data['Date'] >= firstDayOfMonth]
# current_month_data

current_month_data = workout_data.query(" Date >= @firstDayOfMonth ")

In [33]:
current_month_data

,Date,Exercise,Muscle_group,Sets,Reps,Weight,Duration,Note
1786,01/02/2023,Chin Up,Biceps,1.0,5.0,NaN,NaN,NaN
1787,01/02/2023,Chin Up,Biceps,2.0,5.0,NaN,NaN,NaN
1788,01/02/2023,Dips,Triceps,1.0,10.0,NaN,NaN,NaN
1789,01/02/2023,Dips,Triceps,2.0,15.0,NaN,NaN,NaN
1790,01/02/2023,Dips,Triceps,3.0,15.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1782,12/30/2022,Dips,Triceps,3.0,10.0,NaN,NaN,NaN
1783,12/30/2022,Dips,Triceps,4.0,10.0,NaN,NaN,NaN
1775,12/30/2022,Note,Note,NaN,NaN,NaN,NaN,"Went to gym, forgot to record workout."
1776,12/30/2022,Pull Up,Back,1.0,7.0,NaN,NaN,NaN


In [34]:
current_month_data["Date"].unique()

array(['01/02/2023', '04/04/2022', '04/06/2022', '04/09/2022',
       '04/11/2022', '04/12/2022', '04/19/2022', '04/23/2022',
       '04/24/2022', '04/25/2022', '04/26/2022', '04/27/2022',
       '04/28/2022', '05/01/2022', '05/02/2022', '05/03/2022',
       '05/05/2022', '05/06/2022', '05/08/2022', '05/12/2022',
       '05/18/2022', '05/26/2022', '05/30/2022', '05/31/2022',
       '06/02/2022', '06/06/2022', '06/07/2022', '06/09/2022',
       '06/10/2022', '06/13/2022', '06/14/2022', '06/16/2022',
       '06/19/2022', '06/20/2022', '06/21/2022', '06/23/2022',
       '06/26/2022', '07/05/2022', '07/06/2022', '07/07/2022',
       '07/08/2022', '07/19/2022', '07/20/2022', '07/22/2022',
       '07/27/2022', '07/28/2022', '08/02/2022', '08/04/2022',
       '08/08/2022', '08/11/2022', '08/14/2022', '08/19/2022',
       '08/21/2022', '08/24/2022', '08/30/2022', '08/31/2022',
       '09/06/2022', '09/08/2022', '09/13/2022', '09/20/2022',
       '09/22/2022', '09/28/2022', '09/29/2022', '10/03

In [35]:
current_month_data

,Date,Exercise,Muscle_group,Sets,Reps,Weight,Duration,Note
1786,01/02/2023,Chin Up,Biceps,1.0,5.0,NaN,NaN,NaN
1787,01/02/2023,Chin Up,Biceps,2.0,5.0,NaN,NaN,NaN
1788,01/02/2023,Dips,Triceps,1.0,10.0,NaN,NaN,NaN
1789,01/02/2023,Dips,Triceps,2.0,15.0,NaN,NaN,NaN
1790,01/02/2023,Dips,Triceps,3.0,15.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1782,12/30/2022,Dips,Triceps,3.0,10.0,NaN,NaN,NaN
1783,12/30/2022,Dips,Triceps,4.0,10.0,NaN,NaN,NaN
1775,12/30/2022,Note,Note,NaN,NaN,NaN,NaN,"Went to gym, forgot to record workout."
1776,12/30/2022,Pull Up,Back,1.0,7.0,NaN,NaN,NaN


In [36]:
# For loop over list of unique dates in curentMonthData dates's column
# Counting each loop as a Record Day
daysCount = 0
for date in current_month_data[current_month_data['Date'] >= firstDayOfMonth]['Date'].unique():
    print(date)
    daysCount += 1

01/02/2023
04/04/2022
04/06/2022
04/09/2022
04/11/2022
04/12/2022
04/19/2022
04/23/2022
04/24/2022
04/25/2022
04/26/2022
04/27/2022
04/28/2022
05/01/2022
05/02/2022
05/03/2022
05/05/2022
05/06/2022
05/08/2022
05/12/2022
05/18/2022
05/26/2022
05/30/2022
05/31/2022
06/02/2022
06/06/2022
06/07/2022
06/09/2022
06/10/2022
06/13/2022
06/14/2022
06/16/2022
06/19/2022
06/20/2022
06/21/2022
06/23/2022
06/26/2022
07/05/2022
07/06/2022
07/07/2022
07/08/2022
07/19/2022
07/20/2022
07/22/2022
07/27/2022
07/28/2022
08/02/2022
08/04/2022
08/08/2022
08/11/2022
08/14/2022
08/19/2022
08/21/2022
08/24/2022
08/30/2022
08/31/2022
09/06/2022
09/08/2022
09/13/2022
09/20/2022
09/22/2022
09/28/2022
09/29/2022
10/03/2022
10/04/2022
10/05/2022
10/06/2022
10/09/2022
10/10/2022
10/11/2022
10/12/2022
10/13/2022
10/16/2022
10/17/2022
10/18/2022
10/20/2022
10/22/2022
10/25/2022
10/27/2022
10/29/2022
10/30/2022
11/07/2022
11/08/2022
11/10/2022
11/12/2022
11/13/2022
11/14/2022
11/15/2022
11/16/2022
11/19/2022
11/20/2022

In [37]:
lastRecordDate = current_month_data['Date'].unique()[-1]
lastRecordData = current_month_data[current_month_data['Date'] == lastRecordDate]

lastRecordData

,Date,Exercise,Muscle_group,Sets,Reps,Weight,Duration,Note
1778,12/30/2022,Chin Up,Biceps,1.0,7.0,NaN,NaN,NaN
1779,12/30/2022,Chin Up,Biceps,2.0,5.0,NaN,NaN,NaN
1780,12/30/2022,Dips,Triceps,1.0,15.0,NaN,NaN,NaN
1781,12/30/2022,Dips,Triceps,2.0,10.0,NaN,NaN,NaN
1782,12/30/2022,Dips,Triceps,3.0,10.0,NaN,NaN,NaN
1783,12/30/2022,Dips,Triceps,4.0,10.0,NaN,NaN,NaN
1775,12/30/2022,Note,Note,NaN,NaN,NaN,NaN,"Went to gym, forgot to record workout."
1776,12/30/2022,Pull Up,Back,1.0,7.0,NaN,NaN,NaN
1777,12/30/2022,Pull Up,Back,2.0,5.0,NaN,NaN,NaN


In [38]:
# what muscle did we work out last
for unique_muscle in lastRecordData['Muscle_group'].unique():
    print(unique_muscle)

Biceps
Triceps
Note
Back
